In [2]:
#r "nuget: CsvHelper"

Installed Packages CsvHelper, 33.0.1

In [3]:
using System;
using System.IO;
using System.Linq;
using System.Collections.Generic;
using CsvHelper;
using CsvHelper.Configuration;
using Newtonsoft.Json;
using System.Text.Json;  // For JSON serialization/deserialization
using System.Text.Json.Serialization;

In [4]:
 # load "../FeedbackApi/Models/CSVFeedbackRecord.cs"

In [5]:

public class ServiceHighlight
{
    [JsonPropertyName("ServiceName")]
    public string ServiceName { get; set; }

    [JsonPropertyName("TotalFeedback")]
    public int TotalFeedback { get; set; }

    [JsonPropertyName("DistinctCustomers")]
    public int DistinctCustomers { get; set; }

    [JsonPropertyName("FeatureRequests")]
    public int FeatureRequests { get; set; }

    [JsonPropertyName("Bugs")]
    public int Bugs { get; set; }

    [JsonPropertyName("OverallSentiment")]
    public string OverallSentiment { get; set; }
}

## Creating the summary

- Per csv file, we run this short summarization based on meta data

In [7]:
// var filePath = "/Users/yoavdobrin/workspace/fta/feedback-ai-lens/sample-data/cosmosdb.csv";
// var filePath = "/Users/yoavdobrin/workspace/fta/feedback-ai-lens/sample-data/adf.csv";
var filePath = "/Users/yoavdobrin/workspace/fta/feedback-ai-lens/sample-data/aks.csv";
var config = new CsvConfiguration(System.Globalization.CultureInfo.InvariantCulture)
{
    HasHeaderRecord = true,
};

using (var reader = new StreamReader(filePath))
using (var csv = new CsvReader(reader, config))
{
    var records = csv.GetRecords<CSVFeedbackRecord>().ToList();

    // Group by ServiceTree_Name and calculate metrics
    var serviceHighlights = records
        .GroupBy(r => r.ServiceName)
        .Select(g => new ServiceHighlight
        {
            ServiceName = g.Key,
            TotalFeedback = g.Count(),
            DistinctCustomers = g.Select(r => r.CustomerTpid).Distinct().Count(),
            FeatureRequests = g.Count(r => r.Type == "Feature Request"),
            Bugs = g.Count(r => r.Type == "Bug"),
            OverallSentiment = g.Count(r => r.Type == "Feature Request") > g.Count(r => r.Type == "Bug") ? "Positive" : "Neutral"
        })
        .ToList();

    // Serialize the result to JSON
    var json = JsonConvert.SerializeObject(serviceHighlights, Formatting.Indented);
    Console.WriteLine(json);
}

[
  {
    "ServiceName": "Azure Kubernetes Service",
    "TotalFeedback": 600,
    "DistinctCustomers": 192,
    "FeatureRequests": 39,
    "Bugs": 5,
    "OverallSentiment": "Positive"
  }
]
